In [1]:
import sys
sys.path.insert(0, "../src")

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import gc
import warnings
from pathlib import Path
from tqdm.notebook import tqdm

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import pytorch_lightning as pl

import torchvision.transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2

from transformers import AdamW, get_cosine_schedule_with_warmup

import optim
import loss
from utils import visualize, radar2precipitation, seed_everything

<IPython.core.display.Javascript object>

In [4]:
warnings.simplefilter("ignore")

<IPython.core.display.Javascript object>

# U-Net

## Config

In [5]:
args = dict(
    seed=42,
    dams=(6071, 6304, 7026, 7629, 7767, 8944, 11107),
    train_folds_csv=Path("../input/train_folds.csv"),
    train_data_path=Path("../input/train"),
    test_data_path=Path("../input/test"),
    model_dir=Path("../models"),
    output_dir=Path("../output"),
    rng=255.0,
    num_workers=4,
    gpus=1,
    lr=1e-3,
    max_epochs=30,
    batch_size=256,
    precision=16,
    optimizer="adamw",
    scheduler="cosine",
    accumulate_grad_batches=1,
    gradient_clip_val=5.0,
    warmup_epochs=1,
)

args["trn_tfms"] = A.Compose(
    [
        A.PadIfNeeded(min_height=128, min_width=128, always_apply=True, p=1),
        ToTensorV2(always_apply=True, p=1),
    ]
)

args["val_tfms"] = A.Compose(
    [
        A.PadIfNeeded(min_height=128, min_width=128, always_apply=True, p=1),
        ToTensorV2(always_apply=True, p=1),
    ]
)

<IPython.core.display.Javascript object>

## Dataset

In [6]:
class NowcastingDataset(Dataset):
    def __init__(self, paths, tfms=None, test=False):
        self.paths = paths
        if tfms is not None:
            self.tfms = tfms
        else:
            self.tfms = A.Compose(
                [
                    A.PadIfNeeded(
                        min_height=128, min_width=128, always_apply=True, p=1
                    ),
                    ToTensorV2(always_apply=True, p=1),
                ]
            )
        self.test = test

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        data = np.load(path)

        augmented = self.tfms(image=data)
        data = augmented["image"]

        x = data[:4, :, :]
        x = x / args["rng"]
        if self.test:
            return x
        else:
            y = data[4, :, :]
            y = y / args["rng"]
            y = y.unsqueeze(0)

            return x, y

<IPython.core.display.Javascript object>

In [7]:
class NowcastingDataModule(pl.LightningDataModule):
    def __init__(
        self,
        train_df=None,
        val_df=None,
        batch_size=args["batch_size"],
        num_workers=args["num_workers"],
        test=False,
    ):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.test = test

    def setup(self, stage="train"):
        if stage == "train":
            train_paths = [
                args["train_data_path"] / fn for fn in self.train_df.filename.values
            ]
            val_paths = [
                args["train_data_path"] / fn for fn in self.val_df.filename.values
            ]
            self.train_dataset = NowcastingDataset(train_paths, tfms=args["trn_tfms"])
            self.val_dataset = NowcastingDataset(val_paths, tfms=args["val_tfms"])
        else:
            test_paths = list(sorted(args["test_data_path"].glob("*.npy")))
            self.test_dataset = NowcastingDataset(test_paths, test=True)

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            sampler=RandomSampler(self.train_dataset),
            pin_memory=True,
            num_workers=self.num_workers,
            drop_last=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=2 * self.batch_size,
            sampler=SequentialSampler(self.val_dataset),
            pin_memory=True,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=2 * self.batch_size,
            sampler=SequentialSampler(self.test_dataset),
            pin_memory=True,
            num_workers=self.num_workers,
        )

<IPython.core.display.Javascript object>

## Model

### Basic

In [8]:
class BasicBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        assert in_ch == out_ch
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
        )

    def forward(self, x):
        return x + self.net(x)

<IPython.core.display.Javascript object>

### Encoder

In [9]:
class DownBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.id_conv = nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=2)
        self.net = nn.Sequential(
            nn.BatchNorm2d(in_ch),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(in_ch),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
        )

    def forward(self, x):
        residual = x
        residual = self.id_conv(residual)
        x = self.net(x)
        return residual + x, x


class Encoder(nn.Module):
    def __init__(self, chs=[4, 64, 128, 256, 512, 1024]):
        super().__init__()
        self.blocks = nn.ModuleList(
            [DownBlock(chs[i], chs[i + 1]) for i in range(len(chs) - 1)]
        )
        self.basic = BasicBlock(chs[-1], chs[-1])

    def forward(self, x):
        feats = []
        for block in self.blocks:
            x, feat = block(x)
            feats.append(feat)
        x = self.basic(x)
        feats.append(x)
        return feats

<IPython.core.display.Javascript object>

### Decoder

In [10]:
class UpBlock(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super().__init__()
        self.id_conv = nn.ConvTranspose2d(
            in_ch + in_ch, out_ch, kernel_size=2, stride=2
        )
        layers = []
        if bilinear:
            layers.append(nn.Upsample(scale_factor=2, mode="nearest"))
        else:
            layers.append(
                nn.ConvTranspose2d(in_ch + in_ch, out_ch, kernel_size=2, stride=2)
            )
        layers.extend(
            [
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(inplace=True),
                nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(inplace=True),
                nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            ]
        )
        self.block = nn.Sequential(*layers)

    def forward(self, x, feat):
        x = torch.cat([x, feat], dim=1)
        residual = x
        residual = self.id_conv(residual)
        x = self.block(x)
        return x + residual


class Decoder(nn.Module):
    def __init__(self, chs=[1024, 512, 256, 128, 64]):
        super().__init__()
        self.blocks = nn.ModuleList(
            [UpBlock(chs[i], chs[i + 1]) for i in range(len(chs) - 1)]
        )

    def forward(self, x, feats):
        for block, feat in zip(self.blocks, feats):
            x = block(x, feat)
        return x

<IPython.core.display.Javascript object>

### U-Net

In [16]:
class UNet(pl.LightningModule):
    def __init__(
        self,
        lr=args["lr"],
        enc_chs=[4, 64, 128, 256, 512, 1024],
        dec_chs=[1024, 512, 256, 128, 64],
        num_train_steps=None,
    ):
        super().__init__()
        self.lr = lr
        self.num_train_steps = num_train_steps
        self.criterion = nn.L1Loss()

        self.tail = BasicBlock(4, enc_chs[0])
        self.encoder = Encoder(enc_chs)
        self.decoder = Decoder(dec_chs)
        self.head = nn.Sequential(
            nn.ConvTranspose2d(dec_chs[-1], 32, kernel_size=2, stride=2, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(32, 1, kernel_size=3, padding=1),
            #             nn.ReLU(inplace=True),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.tail(x)
        feats = self.encoder(x)
        feats = feats[::-1]
        x = self.decoder(feats[0], feats[1:])
        x = self.head(x)
        return x

    def shared_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        return loss, y, y_hat

    def training_step(self, batch, batch_idx):
        loss, y, y_hat = self.shared_step(batch, batch_idx)
        self.log("train_loss", loss)
        for i, param_group in enumerate(self.optimizer.param_groups):
            self.log(f"lr/lr{i}", param_group["lr"])
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        loss, y, y_hat = self.shared_step(batch, batch_idx)
        return {"loss": loss, "y": y.detach(), "y_hat": y_hat.detach()}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        self.log("val_loss", avg_loss)

        y = torch.cat([x["y"] for x in outputs])
        y_hat = torch.cat([x["y_hat"] for x in outputs])

        crop = T.CenterCrop(120)
        y = crop(y)
        y_hat = crop(y_hat)

        batch_size = len(y)
        y = y.detach().cpu().numpy()
        y *= args["rng"]
        y = y.reshape(batch_size, -1)
        y = y[:, args["dams"]]
        y_hat = y_hat.detach().cpu().numpy()
        y_hat *= args["rng"]
        y_hat = y_hat.reshape(batch_size, -1)
        y_hat = y_hat[:, args["dams"]]

        y_true = radar2precipitation(y)
        y_true = np.where(y_true >= 0.1, 1, 0)
        y_true = y_true.ravel()
        y_pred = radar2precipitation(y_hat)
        y_pred = np.where(y_pred >= 0.1, 1, 0)
        y_pred = y_pred.ravel()

        y = y.ravel()
        y_hat = y_hat.ravel()
        mae = metrics.mean_absolute_error(y, y_hat, sample_weight=y_true)
        self.log("mae", mae)

        tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred).ravel()
        csi = tp / (tp + fn + fp)
        self.log("csi", csi)

        comp_metric = mae / (csi + 1e-12)
        self.log("comp_metric", comp_metric)

        print(
            f"Epoch {self.current_epoch} | MAE/CSI: {comp_metric} | MAE: {mae} | CSI: {csi} | Loss: {avg_loss}"
        )

    def configure_optimizers(self):
        # optimizer
        if args["optimizer"] == "adam":
            self.optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        elif args["optimizer"] == "adamw":
            self.optimizer = AdamW(self.parameters(), lr=self.lr)
        elif args["optimizer"] == "radam":
            self.optimizer = optim.RAdam(self.parameters(), lr=self.lr)
        elif args["optimizer"] == "ranger":
            self.optimizer = optim.RAdam(self.parameters(), lr=self.lr)
            self.optimizer = optim.Lookahead(self.optimizer)

        # scheduler
        if args["scheduler"] == "cosine":
            self.scheduler = get_cosine_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=self.num_train_steps * args["warmup_epochs"],
                num_training_steps=self.num_train_steps * args["max_epochs"],
            )
            return [self.optimizer], [{"scheduler": self.scheduler, "interval": "step"}]
        elif args["scheduler"] == "step":
            self.scheduler = torch.optim.lr_scheduler.StepLR(
                self.optimizer, step_size=10, gamma=0.5
            )
            return [self.optimizer], [
                {"scheduler": self.scheduler, "interval": "epoch"}
            ]
        elif args["scheduler"] == "plateau":
            self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, mode="min", factor=0.1, patience=3, verbose=True
            )
            return [self.optimizer], [
                {
                    "scheduler": self.scheduler,
                    "interval": "epoch",
                    "reduce_on_plateau": True,
                    "monitor": "comp_metric",
                }
            ]
        else:
            self.scheduler = None
            return [self.optimizer]

<IPython.core.display.Javascript object>

## Train

In [12]:
seed_everything(args["seed"])
pl.seed_everything(args["seed"])

42

<IPython.core.display.Javascript object>

In [13]:
df = pd.read_csv(args["train_folds_csv"])

<IPython.core.display.Javascript object>

In [14]:
def train_fold(df, fold, lr_find=False):
    train_df = df[df.fold != fold]
    val_df = df[df.fold == fold]

    datamodule = NowcastingDataModule(train_df, val_df)
    datamodule.setup()

    num_train_steps = np.ceil(
        len(train_df) // args["batch_size"] / args["accumulate_grad_batches"]
    )
    model = UNet(num_train_steps=num_train_steps)

    trainer = pl.Trainer(
        gpus=args["gpus"],
        max_epochs=args["max_epochs"],
        precision=args["precision"],
        progress_bar_refresh_rate=50,
        benchmark=True,
    )

    if lr_find:
        lr_finder = trainer.tuner.lr_find(model, datamodule=datamodule)
        fig = lr_finder.plot(suggest=True)
        fig.show()
        return

    print(f"Training fold {fold}...")
    trainer.fit(model, datamodule)

    checkpoint = (
        args["model_dir"]
        / f"unet_sigmoid_fold{fold}_bs{args['batch_size']}_epochs{args['max_epochs']}_lr{model.lr}_{args['optimizer']}_{args['scheduler']}.ckpt"
    )
    trainer.save_checkpoint(checkpoint)
    print("Model saved at", checkpoint)

    del model, trainer, datamodule
    gc.collect()
    torch.cuda.empty_cache()

<IPython.core.display.Javascript object>

In [16]:
# AdamW bs256 lr 1e-3
for fold in range(5):
    train_fold(df, fold)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold 0...



  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | tail      | BasicBlock | 300   
2 | encoder   | Encoder    | 25 M  
3 | decoder   | Decoder    | 17 M  
4 | head      | Sequential | 8 K   


Epoch 0 | MAE/CSI: 110793664383561.66 | MAE: 110.79366438356165 | CSI: 0.0 | Loss: 0.05267500877380371


Epoch 0 | MAE/CSI: 33.186943419342214 | MAE: 25.55001192118522 | CSI: 0.7698814439856134 | Loss: 0.017848094925284386


...

Epoch 26 | MAE/CSI: 18.0624360317926 | MAE: 14.856623087439964 | CSI: 0.8225149177703391 | Loss: 0.01123881060630083


Epoch 27 | MAE/CSI: 18.33757410027865 | MAE: 15.043335066826108 | CSI: 0.8203557888597258 | Loss: 0.0112459110096097


Epoch 28 | MAE/CSI: 18.386441478880272 | MAE: 15.091226520452597 | CSI: 0.8207801676995989 | Loss: 0.011267893016338348


Epoch 29 | MAE/CSI: 18.40432508550994 | MAE: 15.094416588039563 | CSI: 0.8201559425781535 | Loss: 0.011272534728050232

Model saved at ../models/unet_fold4_bs256_epochs30_lr0.001_adamw_cosine.ckpt


<IPython.core.display.Javascript object>

In [ ]:
# AdamW bs256 lr 1e-3 sigmoid
for fold in range(5):
    train_fold(df, fold)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


Training fold 0...



  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | tail      | BasicBlock | 300   
2 | encoder   | Encoder    | 25 M  
3 | decoder   | Decoder    | 17 M  
4 | head      | Sequential | 8 K   


Epoch 0 | MAE/CSI: 167.45782510687246 | MAE: 27.286654537671232 | CSI: 0.16294642857142858 | Loss: 0.4579598009586334


Epoch 0 | MAE/CSI: 28.362450125306825 | MAE: 22.10657577075336 | CSI: 0.7794311024984428 | Loss: 0.020207127556204796


...

Epoch 6 | MAE/CSI: 22.04263144863797 | MAE: 17.724101042939697 | CSI: 0.8040828103585083 | Loss: 0.01200629211962223


Epoch 7 | MAE/CSI: 21.346540554610538 | MAE: 17.230123555188904 | CSI: 0.8071623367303 | Loss: 0.011888713575899601


Epoch 8 | MAE/CSI: 19.32465185410075 | MAE: 15.690600147796456 | CSI: 0.8119473647566664 | Loss: 0.01206459105014801


Epoch 9 | MAE/CSI: 20.710088985666317 | MAE: 16.81065647000772 | CSI: 0.8117133867276888 | Loss: 0.011814854107797146


Epoch 10 | MAE/CSI: 24.743819129015346 | MAE: 19.77880616991625 | CSI: 0.7993433053630062 | Loss: 0.012558660469949245


## Inference

In [ ]:
def inference(checkpoints):
    datamodule = NowcastingDataModule()
    datamodule.setup("test")
    
    test_paths = datamodule.test_dataset.paths
    test_filenames = [path.name for path in test_paths]
    final_preds = np.zeros((len(datamodule.test_dataset), 14400))
    
    for checkpoint in checkpoints:
        print(f"Inference from {checkpoint}")
        model = UNet.load_from_checkpoint(str(checkpoint))
        model.cuda()
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in tqdm(datamodule.test_dataloader()):
                batch = batch.cuda()
                imgs = model(batch)
                imgs = imgs.detach().cpu().numpy()
                imgs = imgs[:, 0, 4:124, 4:124]
                imgs = args["rng"] * imgs
                imgs = imgs.clip(0, 255)
                imgs = imgs.round()
                preds.append(imgs)
                
        preds = np.concatenate(preds)
        preds = preds.astype(np.uint8)
        preds = preds.reshape(-1, 14400)
        final_preds += preds / len(checkpoint)
        
        del model
        gc.collect()
        torch.cuda.empty_cache()
        
    final_preds = final_preds.round()
    final_preds = final_preds.astype(np.uint8)
    
    subm = pd.DataFrame()
    subm["file_name"] = test_filename
    for i in tqdm(range(14400)):
        subm[str(i)] = final_preds[:, i]
        
    return subm

In [ ]:
checkpoints = [args["model_dir"] / f"unet_fold{fold}_bs{args['batch_size']}_epochs{args['max_epochs']}_lr{model.lr}_{args['optimizer']}_{args['scheduler']}.ckpt" for fold in range(5)]
output_path = args["output_dir"] / f"unet_bs{args['batch_size']}_epochs{args['max_epochs']}_lr{model.lr}_{args['optimizer']}_{args['scheduler']}.csv"
subm.to_csv(output_path, index=False)
subm.head()